# This is  a research notebook to try to find correlation between hyperwave and volume 

In [45]:
# !pip install --upgrade pip

In [76]:
from datetime import datetime
from scipy.spatial import ConvexHull

import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

## This is the place where you setup the symbol and phases

In [71]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'symbol':'AMZN',
}

data_Netflix= {
    'name':'Netflox',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'symbol':'AAPL',
}


In [72]:
company_setup = data_Apple
name = company_setup ['name'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 1)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [49]:
def get_raw_historical_data(symbol, timeframe='daily'):
    url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(symbol, timeframe)
    print("Url to fetch the data {}".format(url_symbol))
    print("fetching data for {} on {}...".format(symbol, timeframe))
    df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
    df_price = df_list[0].dropna()
    return df_price


Url to fetch the data https://www.investopedia.com/markets/api/partial/historical/?Symbol=AAPL&Type=Historical+Prices&Timeframe=Weekly&StartDate=Jan+01%2C+1900
fetching data for AAPL on Weekly...


## Function that allow to calculate the weekid from a start date

In [73]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["date"]-base_date).days/7)

def clean_raw_market_data(df):
    df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
    df = df.rename(columns={'Adj. Close':'close'})
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    return df.sort_values(by="date")

def get_weekId_max_price(df):
    return df.ix[ df['close'].argmax()]["weekId"]


In [67]:
def get_weekId_cartesian_product(df):
    df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
    df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [74]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_raw_historical_data(symbol, 'Weekly')

df_weekly_price = clean_raw_market_data(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)
df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId < max_price_weekId)]
weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

Url to fetch the data https://www.investopedia.com/markets/api/partial/historical/?Symbol=AAPL&Type=Historical+Prices&Timeframe=Weekly&StartDate=Jan+01%2C+1900
fetching data for AAPL on Weekly...


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [75]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])


In [62]:
hull = ConvexHull(df_weekly_price_until_max[['weekId', 'close']])


In [63]:
print(df_weekly_price_until_max)

              Date    Open    High     Low   close       Volume       date  \
1497  Jan 05, 1990    1.12    1.13    1.09    1.12  123379200.0 1990-01-05   
1496  Jan 12, 1990    1.01    1.03    1.00    1.02  172200000.0 1990-01-12   
1495  Jan 19, 1990    1.00    1.02    0.99    1.01  265596800.0 1990-01-19   
1494  Jan 26, 1990    1.01    1.01    0.95    0.97  181776000.0 1990-01-26   
1493  Feb 02, 1990    0.98    1.03    0.98    1.01  118966400.0 1990-02-02   
1492  Feb 09, 1990    0.99    1.02    0.98    1.01  168123200.0 1990-02-09   
1491  Feb 16, 1990    1.02    1.02    1.00    1.00  127579200.0 1990-02-16   
1490  Feb 23, 1990    0.97    0.99    0.97    0.99  150516800.0 1990-02-23   
1489  Mar 02, 1990    0.99    1.03    0.99    1.00  105313600.0 1990-03-02   
1488  Mar 09, 1990    1.09    1.11    1.08    1.09  230966400.0 1990-03-09   
1487  Mar 16, 1990    1.19    1.21    1.16    1.19  645187200.0 1990-03-16   
1486  Mar 23, 1990    1.22    1.28    1.22    1.25  228345600.0 

In [64]:
import plotly.plotly as py
import plotly.graph_objs as go

x = df_weekly_price_until_max['weekId'].values
y =  df_weekly_price_until_max['close'].values
trace0 = go.Scatter(
    x = x,
    y = y,
    mode = 'lines',
    name = 'lines'
)

layout = {
#     'xaxis': {
#         'range': [0, 7]
#     },
#     'yaxis': {
#         'range': [0, 2.5]
#     },
    'shapes': [{
            'type': 'line',
            'x0': x[pair[0]],
            'y0': y[pair[0]],
            'x1': x[pair[1]],
            'y1': y[pair[1]],
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 3,
            }
        } for pair in hull.simplices
        # Line Vertical
        ]
}
data = [trace0]

# py.iplot(data, filename='line-mode')
fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='shapes-lines')

In [65]:
hull.simplices

array([[1492,    0],
       [1000, 1225],
       [   3,   39],
       [ 391,  416],
       [ 391,   39],
       [1375, 1225],
       [1375, 1381],
       [   1,    0],
       [   1,    3],
       [1490, 1492],
       [1490, 1477],
       [1404, 1381],
       [1404, 1477],
       [ 693,  416],
       [ 693,  728],
       [ 993, 1000],
       [ 993,  763],
       [ 761,  763],
       [ 737,  728],
       [ 737,  761]], dtype=int32)

In [53]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)

# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_values(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)

# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
